In [64]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense, ReLU
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.linear_model import ElasticNetCV
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict
from sklearn.linear_model import RidgeCV, Ridge
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn import svm, linear_model
from sklearn.metrics import confusion_matrix,mean_absolute_error,mean_squared_error,mean_squared_log_error,r2_score
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hackerearth-carnival-wars-challenge/sample_submission.csv
/kaggle/input/hackerearth-carnival-wars-challenge/train.csv
/kaggle/input/hackerearth-carnival-wars-challenge/test.csv


In [65]:
data = pd.read_csv("/kaggle/input/hackerearth-carnival-wars-challenge/train.csv")

In [66]:
data.head(5)

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price
0,BRAE2NF6JA5GUEXG,37.0,2015-08-22 18:36:12.000,2,Lillyann,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700
1,TUNE8SFB6RJN2HSD,38.0,2016-03-27 21:19:13.000,24,Klynn,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256
2,BRAEAR7WZPQGPBZU,9.0,2015-08-18 19:25:22.000,447,Ridge,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362
3,WATDZ2ZQ8JPDHCTJ,50.0,2016-03-28 21:53:01.000,23,Abran,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824
4,JWSEBUKYQPMBZ3RK,7.0,2016-03-29 22:58:53.000,63,Dustyn,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935


In [67]:
data=data.drop(['Stall_no', 'Customer_name', 'Product_id'],axis=1)
data.head(5)

,instock_date,Market_Category,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price
0,2015-08-22 18:36:12.000,2,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700
1,2016-03-27 21:19:13.000,24,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256
2,2015-08-18 19:25:22.000,447,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362
3,2016-03-28 21:53:01.000,23,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824
4,2016-03-29 22:58:53.000,63,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935


In [68]:
data_y = data[['Selling_Price']].copy()
data_y

,Selling_Price
0,4185.947700
1,9271.490256
2,6785.701362
3,13028.917824
4,906.553935
...,...
6363,10277.520192
6364,4501.837200
6365,10218.878775
6366,5359.493997


In [69]:
data_x =  data[['Market_Category','Loyalty_customer','Product_Category','Grade','Demand','Discount_avail','charges_1','charges_2 (%)','Minimum_price','Maximum_price']].copy()


In [70]:
data_x


,Market_Category,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price
0,2,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0
1,24,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0
2,447,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0
3,23,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0
4,63,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0
...,...,...,...,...,...,...,...,...,...,...
6363,24,Yes,Technology,0,47,0.0,614.0,10.0,8174.0,11266.0
6364,55,Yes,Child_care,0,35,0.0,269.0,5.0,3564.0,4920.0
6365,358,No,Repair,3,29,0.0,283.0,3.0,5303.0,14614.0
6366,452,Yes,Repair,0,8,0.0,267.0,3.0,4334.0,5849.0


In [71]:
lb_make = LabelEncoder()
data_x["Product_Category"] = lb_make.fit_transform(data_x["Product_Category"])
data_x.head(5)

,Market_Category,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price
0,2,Yes,3,1,68,0.0,376.0,11.0,2983.0,4713.0
1,24,Yes,3,0,51,0.0,397.0,12.0,7495.0,10352.0
2,447,Yes,0,0,10,0.0,250.0,9.0,5752.0,7309.0
3,23,Yes,2,2,48,0.0,144.0,13.0,5090.0,20814.0
4,63,Yes,8,1,35,1.0,211.0,4.0,2430.0,9261.0


In [72]:
lb_make = LabelEncoder()
data_x["Loyalty_customer"] = lb_make.fit_transform(data_x["Loyalty_customer"])
data_x.head(5)

,Market_Category,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price
0,2,1,3,1,68,0.0,376.0,11.0,2983.0,4713.0
1,24,1,3,0,51,0.0,397.0,12.0,7495.0,10352.0
2,447,1,0,0,10,0.0,250.0,9.0,5752.0,7309.0
3,23,1,2,2,48,0.0,144.0,13.0,5090.0,20814.0
4,63,1,8,1,35,1.0,211.0,4.0,2430.0,9261.0


In [73]:
data_x = data_x.fillna(0)
data_y = data_y.fillna(0)


In [74]:
x_train,x_test,y_train,y_test=train_test_split(data_x,data_y,test_size=0.20, random_state=42,shuffle=True)

In [75]:
linear = LinearRegression().fit(x_train,y_train)

In [76]:
predict1=linear.predict(x_test)

In [77]:
print(mean_squared_error(y_test,predict1))
print(mean_absolute_error(y_test,predict1))
print(r2_score(y_test,predict1)*100)
print(linear.score(x_test,y_test)*100)

1146755.207881906
642.4210938356111
86.7496532502719
86.7496532502719


In [78]:
regr = svm.SVR()

In [79]:
regr.fit(x_train,y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVR()

In [80]:
predict2 = regr.predict(x_test)

In [81]:
print(mean_squared_error(y_test,predict2))
print(mean_absolute_error(y_test,predict2))
print(r2_score(y_test,predict2)*100)
print(regr.score(x_test,y_test)*100)

6781041.802952533
1873.9492214310862
21.647484488445944
21.647484488445944


In [82]:
clf = linear_model.Lasso(alpha=0.9, max_iter=2000)

In [83]:
clf.fit(x_train,y_train)

Lasso(max_iter=2000)

In [84]:
predict3 = clf.predict(x_test)

In [85]:
print(mean_squared_error(y_test,predict3))
print(mean_absolute_error(y_test,predict3))
print(r2_score(y_test,predict3)*100)
print(clf.score(x_test,y_test)*100)

1146738.5366215878
642.0497717488604
86.7498458807291
86.7498458807291


In [23]:
rf = RandomForestRegressor(random_state=100, n_estimators=10000)

In [24]:
rf.fit(x_train,y_train)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


KeyboardInterrupt: 

In [ ]:
predict4 = rf.predict(x_test)

In [ ]:
print(mean_squared_error(y_test,predict4))
print(mean_absolute_error(y_test,predict4))
print(r2_score(y_test,predict4)*100)
print(rf.score(x_test,y_test)*100)

In [25]:
test = pd.read_csv("/kaggle/input/hackerearth-carnival-wars-challenge/test.csv")

In [26]:
test.head(5)

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price
0,SCHE4YSTDVPVZVXW,39.0,2016-01-13 07:45:08.000,205,Ivanka,No,Fashion,0,90,0,380.0,16.0,2576.0,3340
1,ACCEGCATKHNRXUHW,49.0,2015-08-23 20:37:05.000,3,Isaak,Yes,Fashion,0,87,0,393.0,16.0,1202.0,1955
2,NKCE6GJ5XVJDXNNZ,1.0,2015-11-14 18:12:39.000,183,Analiese,No,Technology,2,55,0,493.0,9.0,7175.0,15715
3,NKCEB8BK3ZXDHDHM,8.0,2015-11-21 04:56:19.000,358,Rusty,Yes,Child_care,3,86,0,303.0,16.0,5404.0,13078
4,TOPEFDXSAHRNPF94,33.0,2015-05-06 15:09:46.000,167,Eloise,No,Technology,0,27,0,567.0,16.0,4069.0,6244


In [27]:
product_id=test["Product_id"]

In [28]:
test = test.drop(['Stall_no', 'Customer_name', 'Product_id','instock_date' ],axis=1)
test.head(5)

,Market_Category,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price
0,205,No,Fashion,0,90,0,380.0,16.0,2576.0,3340
1,3,Yes,Fashion,0,87,0,393.0,16.0,1202.0,1955
2,183,No,Technology,2,55,0,493.0,9.0,7175.0,15715
3,358,Yes,Child_care,3,86,0,303.0,16.0,5404.0,13078
4,167,No,Technology,0,27,0,567.0,16.0,4069.0,6244


In [29]:
test["Loyalty_customer"] = lb_make.fit_transform(test["Loyalty_customer"])
test["Product_Category"] = lb_make.fit_transform(test["Product_Category"])
test.head(5)

,Market_Category,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price
0,205,0,3,0,90,0,380.0,16.0,2576.0,3340
1,3,1,3,0,87,0,393.0,16.0,1202.0,1955
2,183,0,9,2,55,0,493.0,9.0,7175.0,15715
3,358,1,0,3,86,0,303.0,16.0,5404.0,13078
4,167,0,9,0,27,0,567.0,16.0,4069.0,6244


In [30]:
test = test.fillna(0)

In [31]:
predict4=rf.predict(test)
result=pd.DataFrame({"Product_id":product_id,"Selling_Price":predict4})
result["Selling_Price"]=result["Selling_Price"].apply(abs)
result.to_csv("Result4_2.csv", index=False)

IndexError: list index out of range

In [ ]:
result.head()

In [32]:
predict_linear=clf.predict(test)
result_linear=pd.DataFrame({"Product_id":product_id,"Selling_Price":predict_linear})
result_linear["Selling_Price"]=result_linear["Selling_Price"].apply(abs)
result_linear.to_csv("Result_linear.csv", index=False)

In [52]:
predict_linear

array([ 3785.14472899,  2532.02619639, 10653.47260693, ...,
        4771.63281928,  3633.89253549,  2253.61861443])

In [33]:
dt = DecisionTreeRegressor(max_depth=7)

In [34]:
dt.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=7)

In [35]:
predictdt=dt.predict(x_test)

In [ ]:
print(mean_squared_error(y_test,predictdt))
print(mean_absolute_error(y_test,predictdt))
print(r2_score(y_test,predictdt)*100)
print(dt.score(x_test,y_test)*100)

In [ ]:
predict_dt=dt.predict(test)
result_dt=pd.DataFrame({"Product_id":product_id,"Selling_Price":predict_dt})
result_dt["Selling_Price"]=result_dt["Selling_Price"].apply(abs)
result_dt.to_csv("Result_dt.csv", index=False)

In [36]:
rr = Ridge()
rr.fit(x_train, y_train)

Ridge()

In [53]:
predict6 = rr.predict(x_test)
predict6

array([[9605.92675752],
       [2874.4524064 ],
       [6520.28327398],
       ...,
       [5006.95556147],
       [5461.24142557],
       [7522.04819776]])

In [42]:
print(mean_squared_error(y_test,predict6))
print(mean_absolute_error(y_test,predict6))
print(r2_score(y_test,predict6)*100)
print(rr.score(x_test,y_test)*100)

1146863.130844126
642.1482825031458
86.74840623899884
86.74840623899884


In [54]:
predict_rr=rr.predict(test)
predict_rr = predict_rr.flatten()
predict_rr

array([ 3782.67147721,  2533.79435842, 10651.04450732, ...,
        4772.20888744,  3635.57962606,  2249.20418854])

In [56]:
result_rr=pd.DataFrame({"Product_id":product_id,"Selling_Price":predict_rr})
result_rr["Selling_Price"]=result_rr["Selling_Price"].apply(abs)
result_rr.to_csv("Result_rr.csv", index=False)

In [61]:
enc = ElasticNetCV(cv=5, random_state=0)
enc.fit(x_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


ElasticNetCV(cv=5, random_state=0)

In [62]:
predict7 = enc.predict(x_test)

In [63]:
print(mean_squared_error(y_test,predict7))
print(mean_absolute_error(y_test,predict7))
print(r2_score(y_test,predict7)*100)
print(enc.score(x_test,y_test)*100)

3620260.898167865
1155.5297465457995
58.16917865687492
58.16917865687492
